In [8]:
import tkinter as tk
from tkinter import scrolledtext
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

class SentimentChatbotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Offline Sentiment Chatbot")
        self.root.geometry("500x500")

        try:
            self.root.iconbitmap("logo.ico")
        except:
            pass 

        # GUI Styling
        self.root.configure(bg="#e6f2ff")
        
        self.chat_display = scrolledtext.ScrolledText(root, state='disabled', wrap='word', bg="#f9f9f9", font=("Segoe UI", 10))
        self.chat_display.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        self.entry = tk.Entry(root, width=80, font=("Segoe UI", 10))
        self.entry.pack(padx=10, pady=(0, 10), side=tk.LEFT, fill=tk.X, expand=True)
        self.entry.bind("<Return>", self.send_message)

        self.send_button = tk.Button(root, text="Send", command=self.send_message)
        self.send_button.pack(padx=(0, 10), pady=(0, 10), side=tk.RIGHT)

        self.display_bot_message("Hi! I’m your offline chatbot. Type something!")

    def analyze_sentiment(self, text):
        blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
        sentiment = blob.sentiment.classification  # 'pos' or 'neg'
        return sentiment

    def generate_response(self, sentiment):
        if sentiment == 'pos':
            return "That's great to hear! 😊"
        elif sentiment == 'neg':
            return "I'm sorry to hear that. Want to talk more about it?"
        else:
            return "Thanks for sharing."

    def display_bot_message(self, message, sentiment='neutral'):
        self.chat_display.configure(state='normal')
        color = {'pos': 'green', 'neg': 'red', 'neutral': 'black'}.get(sentiment, 'black')
        self.chat_display.insert(tk.END, "Bot: " + message + "\n", sentiment)
        self.chat_display.tag_config(sentiment, foreground=color)
        self.chat_display.configure(state='disabled')
        self.chat_display.yview(tk.END)

    def display_user_message(self, message):
        self.chat_display.configure(state='normal')
        self.chat_display.insert(tk.END, "You: " + message + "\n", "user")
        self.chat_display.tag_config("user", foreground="blue")
        self.chat_display.configure(state='disabled')
        self.chat_display.yview(tk.END)

    def send_message(self, event=None):
        user_input = self.entry.get().strip()
        if not user_input:
            return
        self.display_user_message(user_input)
        self.entry.delete(0, tk.END)

        sentiment = self.analyze_sentiment(user_input)
        response = self.generate_response(sentiment)
        self.display_bot_message(response, sentiment)

if __name__ == "__main__":
    root = tk.Tk()
    app = SentimentChatbotApp(root)
    root.mainloop()
